## This tutorial shows how to build custom features in py-boost

### Imports

In [1]:
import os
# Optional: set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.makedirs('../data', exist_ok=True)

import joblib
from sklearn.datasets import make_regression

# simple case - just one class is used
from py_boost import GradientBoosting 

### Generate dummy regression data

In [2]:
%%time
X, y = make_regression(150000, 100, n_targets=10, random_state=42)

# we need non negative targets for this example
y = y - y.min(axis=0)

X_test, y_test = X[:50000], y[:50000]
X, y = X[-50000:], y[-50000:]

CPU times: user 2.34 s, sys: 1.7 s, total: 4.05 s
Wall time: 849 ms


### Custom Loss

As it was mentioned in Tutorial_1, not only string alias is valid value for the loss function, but also the instance of Loss class, which is parent class for all loss function

#### Now let's build our own MSLE (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_log_error.html) loss function

**Note**: Actually we have the built-in MSLE, so you still could use strinng alias for it

In [3]:
import cupy as cp
from py_boost.gpu.losses import Loss, Metric

class CustomRMSLEMetric(Metric):
    """First, let's define eval metric to estimate model quality while training"""
    
    def error(self, y_true, y_pred):
        """
        The simpliest way do define a custom metric is to define .error method
        Just tell py_boost how to calculate error at the each point, for out case it is possible
        If it is not possible (for ex. ROC-AUC), you should define __call__ method
        See the Metric class for the details
        
        At that stage y_true is already in GPU memory, so we use CuPy to handle it.
        Usage is the same as NumPy, just replace np with cp
        
        Note: the metric is calculated against processed input (see CustomMSLELoss below)
        """
        return (cp.log1p(y_true) - cp.log1p(y_pred)) ** 2
    
    def compare(self, v0 ,v1):
        """
        The last required method is .compare
        It should return True if v0 metric value is better than v1, False othewise
        """
        return v0 < v1
    
    def __call__(self, y_true, y_pred, sample_weight=None):
        """
        We also update __call__ method to redefine default reduction with square
        """
        return super().__call__(y_true, y_pred, sample_weight) ** .5


class CustomMSLELoss(Loss):
    """Custom MSLE Implementation"""
    
    def preprocess_input(self, y_true):
        """
        This method defines, how raw target should be processed before the train starts
        We expect y_true has shape (n_samples, n_outputs)
        
        Here we will not do the actual preprocess, but just check if targets are non negative
        
        At that stage y_true is already in GPU memory, so we use CuPy to handle it.
        Usage is the same as NumPy, just replace np with cp
        
        Note: All metrics and losses will be computed with this preprocess target
        """
        assert (y_true >= 0).all()
        return y_true
    
    def postprocess_output(self, y_pred):
        """
        Since we modify the target variable, we also need method, that defines 
        how to process model prediction
        """
        
        return cp.expm1(y_pred)
    
    def get_grad_hess(self, y_true, y_pred):
        """
        This method defines how to calculate gradients and hessians for given loss
        Note that training also supports sample_weight, but its applied outside the loss fn,
        so we don't need to handle it here
        """ 
        # grad should have the same shape as y_pred
        grad = y_pred - cp.log1p(y_true)
        # NOTE: Input could be a matrix in multioutput case!
        # But anyway - hessians are ones for all of them
        # So, we just create (n_samples, 1) array of ones 
        # and after that is will be broadcasted over all outputs
        # grad should have the same shape as y_pred or (n_samples, 1)
        hess = cp.ones((y_true.shape[0], 1), dtype=cp.float32)
        
        return grad, hess

    def base_score(self, y_true):
        """
        One last thing we require to define is base score
        This method defines how to initialize an empty ensemble
        In simplies case it could be just an array of zeros
        But usualy it is better to boost from mean values
        Output shape should be (n_outputs, ) 
        
        Note: y_true is already processed array here
        
        """
        return cp.log1p(y_true).mean(axis=0)
    
    


In [4]:
%%time
model = GradientBoosting(CustomMSLELoss(), CustomRMSLEMetric(), lr=0.01, verbose=100, ntrees=1000)

model.fit(X, y, eval_sets=[{'X': X_test, 'y': y_test},])

[14:33:29] Stdout logging level is INFO.
[14:33:29] GDBT train starts. Max iter 1000, early stopping rounds 100
[14:33:30] Iter 0; Sample 0, score = 0.24603539557907483; 
[14:33:32] Iter 100; Sample 0, score = 0.1742483282481912; 
[14:33:35] Iter 200; Sample 0, score = 0.1342659688820449; 
[14:33:38] Iter 300; Sample 0, score = 0.10731344416487074; 
[14:33:40] Iter 400; Sample 0, score = 0.08782596307881492; 
[14:33:43] Iter 500; Sample 0, score = 0.07353079220891415; 
[14:33:46] Iter 600; Sample 0, score = 0.06300246387723472; 
[14:33:48] Iter 700; Sample 0, score = 0.05525294291245993; 
[14:33:51] Iter 800; Sample 0, score = 0.049518312947738174; 
[14:33:53] Iter 900; Sample 0, score = 0.045306569498698365; 
[14:33:56] Iter 999; Sample 0, score = 0.04221849120683567; 
CPU times: user 29.4 s, sys: 4.47 s, total: 33.9 s
Wall time: 32.2 s


In [5]:
model.predict(X_test).shape

(50000, 10)

### Custom colsample strategy

We could also redefine some other things. Let's see the example of creating our bagging strategy. Most of custom things should be done via Callbak. 

To create callback we should inherit Callbak class. There are 4 methods, that could be redefined:
        - before_train - outputs None
        - before_iteration - outputs None
        - after_train - outputs None
        - after_iteration - outputs bool - if training should be stopped after iteration

    Methods receive build_info - the state dict, that could be accessed and modifier

    Basic build info structure:

    build_info = {
            'data': {
                'train': {
                    'features_cpu': np.ndarray - raw feature matrix,
                    'features_gpu': cp.ndarray - uint8 quantized feature matrix on GPU,
                    'target': y - cp.ndarray - processed target variable on GPU,
                    'sample_weight': cp.ndarray - processed sample_weight on GPU or None,
                    'ensemble': cp.ndarray - current model prediction (with no postprocessing,
                        ex. before sigmoid for logloss) on GPU,
                    'grad': cp.ndarray of gradients on GPU, before first iteration - None,
                    'hess': cp.ndarray of hessians on GPU, before first iteration - None,

                    'last_tree': {
                        'leaves': cp.ndarray - nodes indices of the last trained tree,
                        'preds': cp.ndarray - predictions of the last trained tree,
                    }

                },
                'valid': {
                    'features_cpu' the same as train, but list, each element corresponds each validation sample,
                    'features_gpu': ...,
                    'target': ...,
                    'sample_weight': ...,
                    'ensemble': ...,

                    'last_tree': {
                        'leaves': ...,
                        'preds': ...,
                    }

                }
            },
            'borders': list of np.ndarray - list or quantization borders,
            'model': GradientBoosting - model, that is trained,
            'mempool': cp.cuda.MemoryPool - memory pool used for train, could be used to clean memory to prevent OOM,
            'builder': DepthwiseTreeBuilder - the instance of tree builder, contains training params,

            'num_iter': int, current number of iteration,
            'iter_scores': list of float - list of metric values for all validation sets for the last iteration,
        }


In [6]:
import cupy as cp
from py_boost.callbacks.callback import Callback

class ColumnImportanceSampler(Callback):
    """
    This class implements a sampling strategy, 
    that sample columns in proportion to thier importance at each step
    
    We should implement __call__ method to use it as sampler
    """
    def __init__(self, rate=0.5, smooth=0.1, 
                 update_freq=10, inverse=False):
        """
        
        Args:
            rate: float, sampling rate
            smooth: float, smoothing parameter
            update_freq: int importance update frequency
            inverse: inverse the probability of sampling

        Returns:

        """
        # Custom columnns sampler based on feature importance
        self.rate = rate
        self.smooth = smooth
        self.update_freq = update_freq
        self.inverse = inverse
        
    def before_iteration(self, build_info):
        """
        Define what should be doe before each iteration
        """
        # Update feature importance
        num_iter = build_info['num_iter']
        
        if (num_iter % self.update_freq) == 0:
            # update probabilities with actual importance
            p = build_info['model'].get_feature_importance() + 1e-3
            p = cp.asarray(p) / (p.sum())
            # inverse if needed
            if self.inverse:
                p = 1 - p
                p = p / p.sum()
            # apply smoothing
            self.p = p * (1 - self.smooth) + cp.ones_like(p) * self.smooth / p.shape[0]
            
    def __call__(self):
        """
        Method should return the array of indices, that will be used
        to grow the tree at the current step
        """
        # Sample rows
        n = self.p.shape[0]
        index = cp.random.choice(cp.arange(n, dtype=cp.uint64), 
            size=int(self.rate * n), p=self.p)
        
        return index

In [7]:
# create model with new sampler   
# if we pass new sampler to the colsample argument it will used instead of default
# it will also be added to the callback pipeline automatically
# you should not pass samplers to the callbacks argument

model = GradientBoosting(CustomMSLELoss(), CustomRMSLEMetric(), 
                         colsample=ColumnImportanceSampler(0.5), 
                         lr=0.01, verbose=100, ntrees=1000 )

model.fit(X, y, eval_sets=[{'X': X_test, 'y': y_test},])

[14:33:57] Stdout logging level is INFO.
[14:33:57] GDBT train starts. Max iter 1000, early stopping rounds 100
[14:33:57] Iter 0; Sample 0, score = 0.24644921389665553; 
[14:33:59] Iter 100; Sample 0, score = 0.17590711477798346; 
[14:34:00] Iter 200; Sample 0, score = 0.13484779001390923; 
[14:34:02] Iter 300; Sample 0, score = 0.10826939489014992; 
[14:34:03] Iter 400; Sample 0, score = 0.08943576705705947; 
[14:34:05] Iter 500; Sample 0, score = 0.0753772653073726; 
[14:34:07] Iter 600; Sample 0, score = 0.06446689810058637; 
[14:34:08] Iter 700; Sample 0, score = 0.05593631183289121; 
[14:34:10] Iter 800; Sample 0, score = 0.04973719737069171; 
[14:34:12] Iter 900; Sample 0, score = 0.045194617065396514; 
[14:34:13] Iter 999; Sample 0, score = 0.041950662857688406; 


In [8]:
model.predict(X_test).shape

(50000, 10)